In [1]:
from google.cloud import bigquery
bq = bigquery.Client()

In [2]:
import pandas as pd

In [3]:
q = bq.query(
"""
select * 
from bigquery-public-data.geo_us_boundaries.counties
LIMIT 10
""")
q.to_dataframe()

,geo_id,state_fips_code,county_fips_code,county_gnis_code,county_name,lsad_name,lsad_code,fips_class_code,mtfcc_feature_class_code,csa_fips_code,cbsa_fips_code,met_div_fips_code,functional_status,area_land_meters,area_water_meters,int_point_lat,int_point_lon,int_point_geom,county_geom
0,01011,01,01011,00161531,Bullock,Bullock County,06,H1,G4020,None,None,None,A,1613076054,6038093,32.101759,-85.717261,POINT(-85.7172613 32.1017589),"POLYGON((-85.926296 32.054676, -85.920111 32.0..."
1,01025,01,01025,00161538,Clarke,Clarke County,06,H1,G4020,None,None,None,A,3207494106,36657891,31.685521,-87.818624,POINT(-87.8186244 31.6855211),"POLYGON((-88.080207 31.677064, -88.080058 31.6..."
2,01027,01,01027,00161539,Clay,Clay County,06,H1,G4020,None,None,None,A,1564251834,5285207,33.270400,-85.863525,POINT(-85.8635254 33.2703999),"POLYGON((-85.978793 33.327568, -85.978797 33.3..."
3,01029,01,01029,00161540,Cleburne,Cleburne County,06,H1,G4020,None,None,None,A,1450663786,2354896,33.671964,-85.516126,POINT(-85.5161261 33.6719637),"POLYGON((-85.603465 33.788474, -85.603469 33.7..."
4,01111,01,01111,00161582,Randolph,Randolph County,06,H1,G4020,None,None,None,A,1503615937,9285420,33.296475,-85.464068,POINT(-85.4640679 33.2964753),"POLYGON((-85.648139 33.359095, -85.648165 33.3..."
5,01075,01,01075,00161563,Lamar,Lamar County,06,H1,G4020,None,None,None,A,1566544637,1600679,33.787085,-88.087431,POINT(-88.0874309 33.7870852),"POLYGON((-88.24154 33.802962, -88.241641 33.80..."
6,01019,01,01019,00161535,Cherokee,Cherokee County,06,H1,G4020,None,None,None,A,1433623319,120308342,34.069515,-85.654242,POINT(-85.6542417 34.0695153),"POLYGON((-85.789466 34.086297, -85.789266 34.0..."
7,01023,01,01023,00161537,Choctaw,Choctaw County,06,H1,G4020,None,None,None,A,2365869841,19144469,31.990954,-88.248889,POINT(-88.2488894 31.9909539),"POLYGON((-88.44169 32.146407, -88.441873 32.14..."
8,01119,01,01119,00161585,Sumter,Sumter County,06,H1,G4020,None,None,None,A,2340898915,24634883,32.597481,-88.200057,POINT(-88.2000571 32.5974811),"POLYGON((-88.369055 32.742584, -88.369201 32.7..."
9,01035,01,01035,00161543,Conecuh,Conecuh County,06,H1,G4020,None,None,None,A,2202007855,6632046,31.430926,-86.988722,POINT(-86.9887221 31.4309257),"POLYGON((-87.165662 31.542897, -87.166757 31.5..."


In [4]:
q = bq.query(
"""
select count(*) as num_rows 
from bigquery-public-data.geo_us_boundaries.counties
""")
q.to_dataframe()

,num_rows
0,3233


In [5]:
no_cache = bigquery.QueryJobConfig(use_query_cache=False)

In [6]:
#q1
q1 = bq.query(
"""
select geo_id
from bigquery-public-data.geo_us_boundaries.counties
where lsad_name = 'Dane County'
""", job_config=no_cache)
q1.to_dataframe()['geo_id'][0]

'55025'

In [7]:
#q2
q2 = bq.query(
"""
select state_fips_code, count(*) as num_counties
from bigquery-public-data.geo_us_boundaries.counties
group by state_fips_code
order by num_counties desc limit 5
""", job_config=no_cache)
df = q2.to_dataframe()
pd.Series(df.num_counties.values,index=df.state_fips_code).to_dict()

{'48': 254, '13': 159, '51': 133, '21': 120, '29': 115}

In [8]:
#q3
bill = {}
per_tb = 6.25
bill['q1'] = q1.total_bytes_billed / 1024**4 * per_tb
bill['q2'] = q2.total_bytes_billed / 1024**4 * per_tb
bill

{'q1': 5.9604644775390625e-05, 'q2': 5.9604644775390625e-05}

In [9]:
p8 = bigquery.Dataset("cs544-399400.p8")

In [10]:
bq.create_dataset(p8, exists_ok=True)

Dataset(DatasetReference('cs544-399400', 'p8'))

In [11]:
config = bigquery.LoadJobConfig(source_format="PARQUET", write_disposition="WRITE_TRUNCATE")
source = "gs://cs544_p8_ykim846/hdma-wi-2021.parquet"
job = bq.load_table_from_uri(source, "cs544-399400.p8.hdma", job_config=config)
job.result()

LoadJob<project=cs544-399400, location=US, id=1fef4ee4-ba3c-48cc-8737-11492ee3b528>

In [12]:
#q4
[ds.dataset_id for ds in bq.list_datasets("cs544-399400")] # PASTE your project name

['p8']

In [13]:
wi = bq.query("""
SELECT c.county_name, count(*) as counts
FROM cs544-399400.p8.hdma as h
LEFT JOIN bigquery-public-data.geo_us_boundaries.counties as c ON h.county_code = c.county_fips_code 
GROUP BY c.county_name
ORDER BY counts DESC
LIMIT 10
""").to_dataframe()

In [14]:
wi

,county_name,counts
0,Milwaukee,46570
1,Dane,38557
2,Waukesha,34159
3,Brown,15615
4,Racine,13007
5,Outagamie,11523
6,Kenosha,10744
7,Washington,10726
8,Rock,9834
9,Winnebago,9310


In [15]:
#q5
pd.Series(wi.counts.values, index=wi.county_name).to_dict()

{'Milwaukee': 46570,
 'Dane': 38557,
 'Waukesha': 34159,
 'Brown': 15615,
 'Racine': 13007,
 'Outagamie': 11523,
 'Kenosha': 10744,
 'Washington': 10726,
 'Rock': 9834,
 'Winnebago': 9310}

In [16]:
url = "https://docs.google.com/spreadsheets/d/11UeIBqQylAyNUBsIO54p6WiYJWHayQMfHDbUWq1jGco/"

external_config = bigquery.ExternalConfig("GOOGLE_SHEETS")
external_config.source_uris = [url]
external_config.options.skip_leading_rows = 1
external_config.autodetect = True

table = bigquery.Table(p8.table("applications"))
table.external_data_configuration = external_config

table = bq.create_table(table, exists_ok=True)

In [17]:
app = bq.query("""
SELECT count(*) as count
FROM p8.applications
WHERE income = 100000
LIMIT 10
""").to_dataframe()

In [18]:
#q6
app['count'][0]

26

In [19]:
per = bq.query("""
SELECT c.county_name, COUNT(*) as counts
FROM (
    SELECT *, ST_GEOGPOINT(longitude, latitude) AS loc
    FROM p8.applications
) AS a
CROSS JOIN bigquery-public-data.geo_us_boundaries.counties AS c
WHERE c.state_fips_code = '55' AND ST_Within(a.loc, c.county_geom)
GROUP BY county_name
ORDER BY counts DESC
""").to_dataframe()

In [20]:
#q7
pd.Series(per.counts.values, index=per.county_name).to_dict()

{'Dane': 54,
 'Door': 17,
 'Brown': 6,
 'Columbia': 4,
 'Bayfield': 4,
 'Sheboygan': 4,
 'Oneida': 3,
 'Winnebago': 3,
 'Milwaukee': 3,
 'Walworth': 3,
 'Marinette': 2,
 'Kewaunee': 2,
 'Monroe': 2,
 'Barron': 2,
 'Ashland': 2,
 'Jefferson': 2,
 'Green Lake': 2,
 'Wood': 2,
 'Eau Claire': 1,
 'Outagamie': 1,
 'Douglas': 1,
 'Sauk': 1,
 'Crawford': 1,
 'Ozaukee': 1,
 'Manitowoc': 1,
 'Chippewa': 1,
 'Adams': 1,
 'St. Croix': 1,
 'Dodge': 1,
 'Marathon': 1,
 'Polk': 1}